In [ ]:
%pylab inline

import numpy as np
import pandas as pd
from pandas import DataFrame
import math
import statistics
import matplotlib.pyplot as plt
import matplotlib as mpl
import datetime
from pathlib import Path
import pyarrow.parquet as pq
import utm

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%ls data

In [ ]:
# reads the od.parquet file

input_base = Path('data')

odm = pd.read_parquet(input_base / 'od_political_area__stay60.parquet')
display(odm.head(n=2))

In [ ]:
odm = odm.loc[odm['origin_locationid'].isin(['Amstetten', 'Baden', 'Salzburg', 'Graz (Stadt)', 'Linz', 'Lienz'])]

In [ ]:
# reads the od.csv file

#odmfile = 'd:\\od_political_area__stay60_utm.csv'
odmfile = 'D:\\COVID\\test - latlong.csv'

odm = pd.read_csv (odmfile, encoding="latin1")
display(odm.head(n=2))

In [ ]:
# applies some preprocessing on odm:
#   1- convert to UTM coordinate system 
#   2- change date type and format
#   3- sort the of od file based on date and origin-id

def latlong2utm (row):
  xO, yO, zZ, uO = utm.from_latlon(row['oigin_y_lat'], row['origin_x_long'])
  xD, yD, zD, uD = utm.from_latlon(row['y_lat'], row['x_long'])
  return pd.Series({'xO': xO, 'yO': yO, 'xD': xD, 'yD': yD})

odm = odm.merge(odm.apply(latlong2utm, axis=1), left_index=True, right_index=True).drop(['origin_x_long', 'oigin_y_lat', 'x_long', 'y_lat'], axis=1)

#odm['time_bin'] = pd.to_datetime(odm['time_bin'], format='%m/%d/%Y')
odm['time_bin'] = odm['time_bin'].dt.strftime('%m/%d/%Y').apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y").date())

odm = odm.sort_values(['time_bin', 'origin_locationid'])

In [ ]:
display(odm['time_bin'].min())
display(odm['time_bin'].max())

In [ ]:
# generates list of date-ticks for plots

statrDate = datetime.datetime.strptime('02/16/2020', '%m/%d/%Y').date()
endDate = datetime.datetime.strptime('04/24/2020', '%m/%d/%Y').date()
date = statrDate
dates = []
while date <= endDate:
 dates+= [date]
 date += datetime.timedelta(days=7)

dates_dm = [x.strftime('%m/%d/%Y')[0:5] for x in dates]

In [ ]:
# gets an od matrix and calculates activity space parameters
def calcActSpcPars (df):
   count_sum = df['non_unique_count'].sum()

   x_mean = (df['non_unique_count'] * df['xD']).sum() / count_sum
   y_mean = (df['non_unique_count'] * df['yD']).sum() / count_sum

   df['dx'] = df['xD']-x_mean 
   df['dy'] = df['yD']-y_mean
   df['dist'] = np.sqrt(df['dx']**2+df['dy']**2)

   dist_sum = (df['dist']*df['non_unique_count']).sum()/1000
   dist_mean = dist_sum / count_sum

   sig_x2 = (df['dx']**2 * df['non_unique_count']).sum() / count_sum
   sig_y2 = (df['dy']**2 * df['non_unique_count']).sum() / count_sum
   sig_xy = (df['dx']*df['dy'] * df['non_unique_count']).sum() / count_sum

   a = np.sqrt(sig_x2+sig_y2+np.sqrt((sig_x2-sig_y2)**2 + 4*sig_xy**2)) / 2 / 1000
   b = np.sqrt(sig_x2+sig_y2-np.sqrt((sig_x2-sig_y2)**2 + 4*sig_xy**2)) / 2 / 1000
   teta = - np.arctan(2*sig_xy/(sig_x2-sig_y2)) / 2 / math.pi * 180
   area = math.pi*a*b
   shapeIndxex = 1 - np.sqrt(1-((b/a)**2))  
   
   actSpc_pars = [df.iloc[0].xO, df.iloc[0].yO, count_sum, dist_sum, dist_mean, a, b, teta, area, shapeIndxex]

   return actSpc_pars


# generates plots for activity space parameters
def saveGraphs (df):
    origin_id = df['origin_locationid'].values[0]
#    df = df.sort_values(by=['time_bin'])
    columns = ['Num_Travels', 'Sum_Travels', 'Mean_Travels', ['a', 'b'], 'Area', 'Teta', 'Shape_Index']
    titles = ['Number of Travels', 'Sum of Travels (km)', 'Mean of Travels (km)', 'Axes of Destinations Ellipse (km)', 'Area of Destinations Ellipse ($km^2$)', 'Azimuth of Destinations Ellipse (deg)', 'Shaped Index of Destinations Ellipse']

    fig = plt.figure(figsize = (30,10), dpi = 180)
    fig.suptitle(origin_id, fontsize=18)
    axs = [241, 242, 243, 245, 246, 247, 248]
    for i in range(0,7):
      ax = df.plot(x='time_bin',y=columns[i], ax=fig.add_subplot(axs[i]), grid='True')
      plt.xticks(ticks=dates, labels=dates_dm, horizontalalignment='center', fontsize=10)
      plt.title(titles[i], fontsize=12)
      ax.set_xlabel('', fontsize=12)
#      ax.set_ylabel(titles[i], fontsize=14)
      ax.yaxis.grid(False)
    path = 'd:\\COVID\\outputs\\'+origin_id+'.png'
    plt.savefig(path)
    plt.clf()

In [ ]:
# main
actSpc_pars = odm.groupby(['time_bin', 'origin_locationid'])[['xO', 'yO', 'xD', 'yD', 'non_unique_count']].apply(calcActSpcPars)
df = DataFrame(actSpc_pars)

col = ['X_Origin', 'Y_Origin', 'Num_Travels', 'Sum_Travels', 'Mean_Travels', 'a', 'b', 'Teta', 'Area', 'Shape_Index']
for c in enumerate(col):
#    print(c)
    df[c[1]] = df[0].apply(lambda x: x[c[0]])#.astype(float)
df = df.reset_index().drop([0], axis=1)#.head()
df.to_csv('d:\\COVID\\outputs\\ActivitySpacePars.csv', index=False)

mpl.rc('figure', max_open_warning = 0)
df.groupby(['origin_locationid']).apply(saveGraphs)